# Optimization Tutorial

The best current example for how to use SymForce for optimization from Python is the tutorial example in `README.md`.

For examples of how to run optimization from C++, see `symforce/examples/bundle_adjustment_in_the_large` and `symforce/examples/bundle_adjustment`.

## Point cloud regisration problem
Let's walk through solving a simplified [point cloud registration problem](https://en.wikipedia.org/wiki/Point-set_registration) with Symforce. In this example, we know the correspondences between two point clouds and we would like to find a `sf.Pose3` to align two point clouds.

### Symforce setup 

In this example, we will use symbolic variables to define our residual function so we need to import `symforce.symbolic`.

In [1]:
import symforce
symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")
symforce.set_epsilon_to_symbol()

import symforce.symbolic as sf

We build up input `Values` using runtime types, e.g. `sym.pose3`, `np.ndarray` instead of symbolic types.

In [2]:
from symforce.values import Values
import numpy as np
import sym

For building a factor graph and solve the problem, we need to import `Factor` and `Optimizer`.

In [10]:
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer

In this example, we will also show the usage of customized cost functions for computing residuals. 

In [9]:
from symforce.opt.noise_models import IsotropicNoiseModel

### Set up the problem

We first generate 30 points in robot body frame.

In [4]:
np.random.seed(seed=1024)
num_points = 30
points_b = []
for i in range(num_points):
    points_b.append(np.random.uniform(low=0.0, high=1.0, size=(3,)))

We create a ground truth `w_T_b_GT`, and use it to transform the 30 points in body frame to world frame.  

In [5]:
pose_tangent_delta = np.array([0.1, 0.1, 0, 0, 0, 10])
w_T_b_GT = sym.Pose3.identity().retract(pose_tangent_delta)
points_w = []
for i in range(num_points):
    points_w.append(w_T_b_GT * points_b[i])

### Build an optimization problem

We define a simple residual function which simply calculates the difference between two corresponded points in the aligned point clouds. Note that we apply a `IsotropicNoiseModel` to whiten the residual. It doesn't really affect the optimization results but you can see the final total error is scaled by the `sigma`. 

In [6]:
def projection_residual(w_T_b: sf.Pose3, b_t_p: sf.V3, w_t_p: sf.V3, sigma: sf.Scalar) -> sf.V3:
    noise_model = IsotropicNoiseModel.from_sigma(sigma)
    return noise_model.whiten(w_T_b*b_t_p - w_t_p)

We build up the inputs to the optimization problem using symforce runtime types. Note that the initial pose value is `sym.Pose3.identity()` which is different from `w_T_b_GT`. We hope the optimizer can give us a solution very close to `w_T_b_GT`.

In [7]:
inputs = Values(
    w_T_b=sym.Pose3.identity(),
    points_b=points_b,
    points_w=points_w,
    sigma=10.0,
)

Create `Factor`s  from the residual functions and a set of keys.

In [11]:
factors = []
for i in range(num_points):
    factors.append(
        Factor(
            residual=projection_residual,
            keys=["w_T_b", f"points_b[{i}]", f"points_w[{i}]", "sigma"],
        )
    )

### Solve the optimization problem

We create an Optimizer with factors and tell it to only optimize the pose key (the rest are held constant). Here we show lots of configure parameters for the optimizer you can tune.

In [12]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["w_T_b"],
    debug_stats=False,
    params=Optimizer.Params(
        verbose = False,
        initial_lambda = 1.0,
        lambda_up_factor = 4.0,
        lambda_down_factor = 1 / 4.0,
        lambda_lower_bound = 0.0,
        lambda_upper_bound = 10000.0,
        use_diagonal_damping = False,
        use_unit_damping = False,
        keep_max_diagonal_damping = False,
        diagonal_damping_min = 1e-6,
        iterations = 500,
        early_exit_min_reduction = 1e-6,
        enable_bold_updates = False,
    )
)

Run the optimization! This returns an `Optimizer.Result` object that contains the optimized values and many other things.

In [14]:
results = optimizer.optimize(inputs)

We can print out the pose before and after the optimization and compare it against our `w_T_b_GT`.

In [15]:
print(f"Early exist? : {results.early_exited}")
print("w_T_b_GT: ")
display(w_T_b_GT)
print("Initial w_T_b: ")
display(results.initial_values.attr.w_T_b)
print("Optimized w_T_b: ")
display(results.optimized_values.attr.w_T_b)

Early exist? : True
w_T_b_GT: 


<Pose3 [0.04995834374876001, 0.04995834374876001, 0.0, 0.9975010414930711, 0.0, 0.0, 10.0]>

Initial w_T_b: 


<Pose3 [0, 0, 0, 1, 0, 0, 0]>

Optimized w_T_b: 


<Pose3 [0.049958343748759994, 0.04995834374875999, 5.6387511044753574e-18, 0.997501041493071, 3.670711373690943e-17, -4.4328076914926344e-17, 10.0]>

We can also print out the total error before and after optimization. Try to change the `sigma` to see how it will change the error value. 

In [16]:
initial_linearization = optimizer.linearize(results.initial_values)
print(f"Initial error: {initial_linearization.error()}")

optimized_linearization = optimizer.linearize(results.optimized_values)
print(f"Final error: {optimized_linearization.error()}")

Initial error: 14.95341695550227
Final error: 1.2710806613746568e-31
